In [1]:
import pandas as pd

In [2]:
from rank_bm25 import * 

In [3]:
from rank_bm25 import *

In [4]:
pip install rank-bm25

Note: you may need to restart the kernel to use updated packages.


### Tokenization ###

In [5]:
from nltk.tokenize import word_tokenize
sentence = 'Jack is a sharp minded fellow'
words = word_tokenize(sentence)
print(words)

['Jack', 'is', 'a', 'sharp', 'minded', 'fellow']


### Removing special characters ###

In [6]:
def spl_chars_removal(lst):
    lst1=list()
    for element in lst:
        str=''
        str = re.sub('[⁰-9a-zA-Z]','',element)
        lst1.append(str)
    return lst1

In [7]:
#adding words to stopwords
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS

In [8]:
#adding custom words to the pre-defined stop words list
all_stopwords_gensim = STOPWORDS.union(set(['disease']))

In [9]:
def stopwprds_removal_gensim_custom(lst):
    lst1=list()
    for str in lst:
        text_tokens = word_tokenize(str)
        tokens_without_sw = [word for word in text_tokens if not word in all_stopwords_gensim]
        str_t =' '.join(tokens_without_sw)
        lst1.append(str_t)
 
    return lst1

### Stemming ###

In [10]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer() 
sentence = 'Machine Learning is cool'
for word in sentence.split():
   print(ps.stem(word))

machin
learn
is
cool


In [19]:
tokenized_corpus = [doc.split(' ') for doc in sentence]
bm25 = BM25Okapi(tokenized_corpus)
query = 'vaccine' ## Enter search query
tokenized_query = query.split(' ')

In [20]:
doc_scores = bm25.get_scores(tokenized_query)
print(doc_scores)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [22]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [26]:
docs = bm25.get_top_n(tokenized_query,sentence, n=5)
df_search = df[df['Text'].isin(docs)]
df_search.head()

NameError: name 'df' is not defined

In [12]:
corpus = [
    'Human machine interface for lab abc computer applications',
    'A survey of user opinion of computer system response time',
    'The EPS user interface management system',
    'System and human system engineering testing of EPS',
    'Relation of user perceived response time to error measurement',
    'The generation of random binary unordered trees',
    'The intersection graph of paths in trees',
    'Graph minors IV Widths of trees and well quasi ordering',
    'Graph minors A survey'
]

# remove stop words and tokenize them (we probably want to do some more
# preprocessing with our text in a real world setting, but we'll keep
# it simple here)
stopwords = set(['for', 'a', 'of', 'the', 'and', 'to', 'in'])
texts = [
    [word for word in document.lower().split() if word not in stopwords]
    for document in corpus
]

# build a word count dictionary so we can remove words that appear only once
word_count_dict = {}
for text in texts:
    for token in text:
        word_count = word_count_dict.get(token, 0) + 1
        word_count_dict[token] = word_count

texts = [[token for token in text if word_count_dict[token] > 1] for text in texts]
texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [17]:
class BM25:
     def __init__(self, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1

     def fit(self, corpus):
        tf = []
        df = {}
        idf = {}
        doc_len = []
        corpus_size = 0
        for document in corpus:
            corpus_size += 1
            doc_len.append(len(document))

            # compute tf (term frequency) per document
            frequencies = {}
            for term in document:
                term_count = frequencies.get(term, 0) + 1
                frequencies[term] = term_count

            tf.append(frequencies)

            # compute df (document frequency) per term
            for term, _ in frequencies.items():
                df_count = df.get(term, 0) + 1
                df[term] = df_count

        for term, freq in df.items():
            idf[term] = math.log(1 + (corpus_size - freq + 0.5) / (freq + 0.5))

        self.tf_ = tf
        self.df_ = df
        self.idf_ = idf
        self.doc_len_ = doc_len
        self.corpus_ = corpus
        self.corpus_size_ = corpus_size
        self.avg_doc_len_ = sum(doc_len) / corpus_size
        return self

     def search(self, query):
        scores = [self._score(query, index) for index in range(self.corpus_size_)]
        return scores

     def _score(self, query, index):
        score = 0.0

        doc_len = self.doc_len_[index]
        frequencies = self.tf_[index]
        for term in query:
            if term not in frequencies:
                continue

            freq = frequencies[term]
            numerator = self.idf_[term] * freq * (self.k1 + 1)
            denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.avg_doc_len_)
            score += (numerator / denominator)

        return score

In [18]:
query = 'The intersection of graph survey and trees'
query = [word for word in query.lower().split() if word not in stopwords]

bm25 = BM25()
bm25.fit(texts)
scores = bm25.search(query)

for score, doc in zip(scores, corpus):
    score = round(score, 3)
    print(str(score) + '\t' + doc)

0.0	Human machine interface for lab abc computer applications
0.999	A survey of user opinion of computer system response time
0.0	The EPS user interface management system
0.0	System and human system engineering testing of EPS
0.0	Relation of user perceived response time to error measurement
1.522	The generation of random binary unordered trees
2.532	The intersection graph of paths in trees
2.167	Graph minors IV Widths of trees and well quasi ordering
2.514	Graph minors A survey
